In [1]:
!pip install nltk

In [2]:
from nltk.tokenize import word_tokenize
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [13]:
vocab = {"<unk>" : 0}
tokens = word_tokenize(document.lower())
# print(len(tokens))
count = Counter(tokens)
# print(len(count))
for word in list(count.keys()) :
  vocab[word] = len(vocab)
print(len(vocab))

289


In [14]:
def text_to_indices(text) :
  tokens = word_tokenize(text.lower())
  indices = []
  for word in tokens :
    if word in vocab :
      indices.append(vocab[word])
    else :
      indices.append(vocab["<unk>"])
  return indices

In [18]:
sentences = document.split('\n')
print(len(sentences))

78


In [28]:
sent_indices = []
max_len = 0
for sent in sentences :
  indices = text_to_indices(sent)
  max_len = max(max_len, len(indices))
  sent_indices.append(indices)
print(max_len)

62


In [32]:
data = []
for sent in sent_indices :
  for i in range(1, len(sent)) :
    data.append(sent[:i+1])
print(len(data))

942


In [36]:
pad_data = []
for sent in data :
  pad_sent = [0] * (max_len - len(sent)) + sent
  pad_data.append(pad_sent)

pad_data = torch.tensor(pad_data, dtype = torch.long)
print(pad_data.shape)

torch.Size([942, 62])


In [37]:
x = pad_data[:, :-1]
y = pad_data[:, -1]
print(x.shape, y.shape)

torch.Size([942, 61]) torch.Size([942])


In [38]:
class docData(Dataset) :
  def __init__(self, x, y) :
    self.x = x
    self.y = y

  def __len__(self) :
    return self.x.shape[0]

  def __getitem__(self, index) :
    return self.x[index], self.y[index]

In [39]:
dataset = docData(x, y)
train_loader = DataLoader(dataset, batch_size = 128, shuffle = True)

In [42]:
class LSTM_model(nn.Module) :
  def __init__(self, vocab_size) :
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 128, batch_first = True)
    self.fc = nn.Linear(128, vocab_size)

  def forward(self, x) :
    out = self.embedding(x)
    _, (hidden_state, cell_state) = self.lstm(out)
    out = self.fc(hidden_state.squeeze(0))
    return out

In [49]:
a = nn.Embedding(289, 100)
b = nn.LSTM(100, 128, batch_first = True)
c = nn.Linear(128, 289)

x = dataset[0][0].unsqueeze(0)
print(x.shape)
y = a(x)
print(y.shape)
z, (hidden_state, cell_state) = b(y)
print(z.shape, hidden_state.shape, cell_state.shape)
out = c(hidden_state.squeeze(0))
print(out.shape)

torch.Size([1, 61])
torch.Size([1, 61, 100])
torch.Size([1, 61, 128]) torch.Size([1, 1, 128]) torch.Size([1, 1, 128])
torch.Size([1, 289])


In [50]:
model = LSTM_model(len(vocab))
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
epochs = 50
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [51]:
for epoch in range(epochs) :
  total_loss = 0.0
  for batch_x, batch_y in train_loader :
    out = model(batch_x)
    loss = loss_fn(out, batch_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch : {epoch} | Loss : {total_loss}")

Epoch : 0 | Loss : 45.12713575363159
Epoch : 1 | Loss : 44.002492904663086
Epoch : 2 | Loss : 41.90834617614746
Epoch : 3 | Loss : 39.60223865509033
Epoch : 4 | Loss : 37.772111892700195
Epoch : 5 | Loss : 36.250927448272705
Epoch : 6 | Loss : 34.950353145599365
Epoch : 7 | Loss : 33.52888107299805
Epoch : 8 | Loss : 32.18020248413086
Epoch : 9 | Loss : 30.92086100578308
Epoch : 10 | Loss : 29.647595167160034
Epoch : 11 | Loss : 28.472394704818726
Epoch : 12 | Loss : 27.11970615386963
Epoch : 13 | Loss : 25.76723575592041
Epoch : 14 | Loss : 24.695905447006226
Epoch : 15 | Loss : 23.492282152175903
Epoch : 16 | Loss : 22.287156581878662
Epoch : 17 | Loss : 21.531165599822998
Epoch : 18 | Loss : 20.111427307128906
Epoch : 19 | Loss : 19.37274932861328
Epoch : 20 | Loss : 18.15273952484131
Epoch : 21 | Loss : 17.398516416549683
Epoch : 22 | Loss : 16.437533855438232
Epoch : 23 | Loss : 15.635674118995667
Epoch : 24 | Loss : 14.73039197921753
Epoch : 25 | Loss : 13.87644076347351
Epoch : 

In [55]:
def predict(model, text, vocab) :
  indices = text_to_indices(text)
  pad_sent = [0] * (max_len - len(indices)) + indices
  pad_sent = torch.tensor(pad_sent, dtype = torch.long).unsqueeze(0)
  out = model(pad_sent)
  _, pred = torch.max(out, dim = 1)
  word = list(vocab.keys())[pred.item()]
  return text + " " + word

In [56]:
sent = "What is the course"
for i in range(5) :
  sent = predict (model, sent, vocab)
  print(sent)

What is the course fee
What is the course fee for
What is the course fee for data
What is the course fee for data science
What is the course fee for data science mentorship
